In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les fichiers CSV depuis le dépôt GitHub (si en ligne)
data_path = "data/"  # Modifier si nécessaire

# Chargement des données
events_df = pd.read_csv(data_path + "cleaned_events.csv")
category_df = pd.read_csv(data_path + "category_tree.csv")

# Aperçu des données
print(events_df.head())
print(events_df.info())
print(events_df.isnull().sum())

# Convertir timestamp en datetime
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'], unit='ms')

# Analyse du nombre d'événements par jour
events_df.set_index('timestamp').resample('D').size().plot(
    title="Nombre d'événements par jour", figsize=(12, 6))
plt.show()

# Distribution des événements
plt.figure(figsize=(8, 4))
sns.countplot(y=events_df['event'], order=events_df['event'].value_counts().index)
plt.title("Répartition des événements")
plt.show()

# Calcul du taux de conversion
conversion_rates = events_df['event'].value_counts(normalize=True) * 100
print(conversion_rates)

# Ajout au panier avant achat
add_to_cart = events_df[events_df['event'] == 'addtocart'].groupby('visitorid').size()
transactions = events_df[events_df['event'] == 'transaction'].groupby('visitorid').size()
cart_vs_purchase = add_to_cart.mean() / transactions.mean()
print(f"Nombre moyen d'ajouts au panier avant un achat : {cart_vs_purchase:.2f}")

# Analyse des jours de la semaine
events_df['day_of_week'] = events_df['timestamp'].dt.dayofweek
events_df.groupby('day_of_week')['event'].value_counts(normalize=True).unstack().plot(
    kind='bar', figsize=(12,6), title="Répartition des événements par jour")
plt.show()

# Meilleurs et pires taux de conversion par produit
views = events_df[events_df['event'] == 'view']['itemid'].value_counts()
purchases = events_df[events_df['event'] == 'transaction']['itemid'].value_counts()
conversion_per_item = (purchases / views).dropna().sort_values(ascending=False)

print("Top 10 produits avec le meilleur taux de conversion:")
print(conversion_per_item.head(10))
print("\nTop 10 produits avec le pire taux de conversion:")
print(conversion_per_item.tail(10))

# Simulation d'un A/B Test sur la couleur d'un bouton
def simulate_ab_test(n=1000, conv_rate_A=0.05, conv_rate_B=0.06):
    group_A = np.random.binomial(1, conv_rate_A, n)
    group_B = np.random.binomial(1, conv_rate_B, n)
    conv_A, conv_B = np.mean(group_A), np.mean(group_B)
    print(f"Conversion A: {conv_A:.3f}, Conversion B: {conv_B:.3f}")
    return conv_A, conv_B

simulate_ab_test()
